## PUR Experiment

In [2]:
## Load in the dMG configuration file with dHBV1.1p options:
import sys
import hydra
from omegaconf import DictConfig, OmegaConf

sys.path.append('../../dMG') # Add the root directory of dMG to the path
from core.utils import initialize_config

## Update the configuration file with PUR train/test options
CONFIG_PATH = '../conf'
CONFIG_NAME = 'dhbv_v1_1p_spatial_config'



def load_config(config: str, config_name: str) -> DictConfig:
    """ Initialize Hydra and parse model configuration yaml(s) into config dict. """
    with hydra.initialize(config_path=config, version_base='1.3'):
        config = hydra.compose(config_name=config_name)
   
    config_dict = OmegaConf.to_container(config, resolve=True)

    initialize_config(config_dict)
    return config_dict

config = load_config(CONFIG_PATH, CONFIG_NAME)

In [ ]:
config[]

In [ ]:
puN = 'PUR'
# Divide CAMELS dataset into 7 continous PUR regions, as shown in Feng et al, 2021 GRL; 2022 HESSD
# get the id list of each PUR region, save to list
regionID = list()
regionNum = list()
# seven regions including different HUCs
regionDivide = [ [1,2], [3,6], [4,5,7], [9,10], [8,11,12,13], [14,15,16,18], [17] ]
for ii in range(len(regionDivide)):
    tempcomb = regionDivide[ii]
    tempregid = list()
    for ih in tempcomb:
        tempid = gageid[hucinfo==ih].tolist()
        tempregid = tempregid + tempid
    regionID.append(tempregid)
    regionNum.append(len(tempregid))

iexp = testfoldInd - 1  #index
TestLS = regionID[iexp] # basin ID list for testing, hold out for training
TestInd = [gageidLst.index(j) for j in TestLS]
TrainLS = list(set(gageid.tolist()) - set(TestLS)) # basin ID for training
TrainInd = [gageidLst.index(j) for j in TrainLS]
gageDic = {'TrainID': TrainLS, 'TestID': TestLS}

## PUB Test

puN = 'PUB'
# load the PUB basin groups
# randomly divide CAMELS basins into 10 groups and this file contains the basin ID for each group
# located in splitPath
splitPath = 'PUBsplitLst.txt'
with open(splitPath, 'r') as fp:
    testIDLst=json.load(fp)
# Generate training ID lists excluding the hold out fold
TestLS = testIDLst[testfoldInd - 1]
TestInd = [gageidLst.index(j) for j in TestLS]
TrainLS = list(set(gageid.tolist()) - set(TestLS))
TrainInd = [gageidLst.index(j) for j in TrainLS]
gageDic = {'TrainID':TrainLS, 'TestID':TestLS}